# CNN Model for QuickDraw Dataset

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras import layers
# from tensorflow.keras import callbacks
# from tensorflow.keras import utils
# from keras import Model, Sequential, layers, regularizers, optimizers
# from sklearn.preprocessing import TargetEncoder
# from colorama import Fore, Style
# import numpy as np
# import pandas as pd
# from typing import Tuple

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import callbacks
from tensorflow.keras.optimizers import Adam
import numpy as np
from typing import Tuple
import time

In [ ]:
# Initialise model
def initialize_model() -> Model:
    '''
    Initialise model with the same CNN structure we used for number recognition - accepting bmp files of dimension 28x28 bits
    * `Conv2D` layer with 8 filters, each of size (4, 4), an input shape of (28x28), the `relu` activation function, and `padding='same'
    * `MaxPool2D` layer with a `pool_size` equal to (2, 2)
    * second `Conv2D` layer with 16 filters, each of size (3, 3), and the `relu` activation function
    * second `MaxPool2D` layer with a `pool_size` equal to (2, 2)
    
    * `Flatten` layer
    * first `Dense` layer with 10 neurons and the `relu` activation function
    * last softmax predictive layer of 50 classes - i.e. number of classes in data subset
    '''

    model = models.Sequential()

    ### First Convolution & MaxPooling
    model.add(layers.Conv2D(8, (4,4), input_shape=(28, 28, 1), activation='relu', padding='same'))
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    
    ### Second Convolution & MaxPooling
    model.add(layers.Conv2D(16, (3,3), activation='relu', padding='same'))
    model.add(layers.MaxPool2D(pool_size=(2,2)))

    ### Flattening
    model.add(layers.Flatten())

    ### One Fully Connected layer - "Fully Connected" is equivalent to saying "Dense"
    model.add(layers.Dense(10, activation='relu'))

    ### Last layer - Classification Layer with 10 outputs corresponding to 10 digits
    model.add(layers.Dense(50, activation='softmax'))
    
    print("✅ Model initialized")

    return model

In [ ]:
def compile_model(model: Model, learning_rate=0.0005) -> Model:
    '''
    Compile the model, which:
    * optimizes the `categorical_crossentropy` loss function,
    * with the `adam` optimizer with learning_rate=0.0005, 
    * and the `accuracy` as the metrics
    '''

    # Create optimizer with custom learning rate
    optimizer = Adam(learning_rate=learning_rate)

    # Compile model
    model.compile(loss="categorical_crossentropy", 
                  optimizer=optimizer, 
                  metrics=["accuracy"])

    print("✅ Model compiled")

    return model

In [ ]:
# Model summary 
model = initialize_model()
model = compile_model(model)
model.summary()

In [ ]:
# Save json to local folder - to store training history
def save_json_to_local (data: list, folder_path: str, file_name: str) -> None: 
    file_path = folder_path+file_name
    with open(file_path, 'w') as file : 
        json.dump(data, file)
    print(f'Saved data to {file_path}')
    return None

In [ ]:
def train_model(
            model: Model,
            X: np.ndarray,
            y: np.ndarray,
            batch_size=256,
            patience=3,
            validation_data=None, # overrides validation_split, if available
            validation_split=0.2
            ) -> Tuple[Model, dict]:
    '''
    Train on the model and return a tuple (fitted_model, history)
    Checkpoints have also been included to store model weights after each epoch
    ''' 
    
    es = callbacks.EarlyStopping(
            monitor="val_accuracy",
            patience=patience,
            restore_best_weights=True,
            verbose=1
            )
    
    # Create checkpoints
    checkpoint_filepath = '/home/jupyter/lewagon_projects/pictionary-ai/raw_data/models_1003_50classes'
    
    #Save the checkpoints in the checkpoint_filepath
    model_checkpoint_callback = callbacks.ModelCheckpoint(
            filepath=checkpoint_filepath,
            save_weights_only=True,
            monitor='val_accuracy',
            mode='max',
            save_best_only=True
            )

    history = model.fit(
            X,
            y,
            validation_data=validation_data,
            validation_split=validation_split,
            epochs=50,
            batch_size=batch_size,
            callbacks=[es, model_checkpoint_callback],
            verbose=1
            )
    
    print(f"✅ Model trained on {len(X)} rows with maximum val accuracy: {round(np.min(history.history['accuracy']), 2)}")

    timestr = time.strftime("%Y%m%d-%H%M%S")
    save_json_to_local (history, checkpoint_filepath, 'model_history_'+timestr)  

    return model, history


In [ ]:
def evaluate_model(
            model: Model,
            X: np.ndarray,
            y: np.ndarray,
            batch_size=64
            ) -> Tuple[Model, dict]:
    '''
    Evaluate performance of the trained model on the test dataset
    Returns evaluation metrics
    '''

    if model is None:
        print(f"\n❌ No model to evaluate")
        return None

    metrics = model.evaluate(
            x=X,
            y=y,
            batch_size=batch_size,
            verbose=0,
            # callbacks=None,
            return_dict=True
            )

    loss = metrics["loss"]
    accuracy = metrics["accuracy"]

    print(f"✅ Model evaluated, accuracy: {round(accuracy, 2)}")

    return metrics
